In [1]:
from music21 import *
import glob

import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook as tqdm

In [4]:
new_ravel_xml = []
file_count = 0
for i,file in tqdm(enumerate(glob.glob("/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/*.mid"))):
    print(file)
    try:
        midi = converter.parse(file)
        xml = midi.write('musicxml','ravel_xml{}.xml'.format(file_count))
        new_ravel_xml.append(xml)
        file_count += 1
    except Exception:
        print('This File did not convert')
        pass
    except NameError:
        print('This File did not convert')
        pass

/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_eau.mid
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_eau_format0.mid
This File did not convert
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_gib.mid
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_gib_format0.mid
This File did not convert
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_ondi.mid
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_ondi_format0.mid
This File did not convert
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_scarbo.mid
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/rav_scarbo_format0.mid
This File did not convert
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/ravel_miroirs_1.mid
/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/midi_files/ravel_miroirs_1_format0.mid



In [5]:
cd ravel_xml

/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/ravel_xml


In [6]:
pwd

'/Volumes/S190813/Music_Box/LTSM_Generative_Music/Ravel/ravel_xml'

In [8]:
with open('ravel_xml_list.txt', 'w') as f:
    for i in glob.glob("*.xml"):
        f.write("%s\n" % i)

In [7]:
from os import listdir
from os.path import isfile, getsize

In [150]:
def parse(mxllist, composer):
    composer_list = [f for f in mxllist if f.replace('-', '_').split('_')[0] == composer]
    for file in tqdm(composer_list):
        s = converter.parse('/Volumes/S190813/Music_box/LTSM_Generative_Music/Ravel/ravel_xml/'+file)
        try:
            k = s.flat.keySignature.sharps
        except AttributeError:
            k = s.analyze('key').sharps
        except:
            with open('{}-parsed.txt'.format(composer), 'a') as output_file:            
                output_file.write('key could not by analyzed\n')
            with open('{}-transposed.txt'.format(composer), 'a') as output_file:            
                output_file.write('key could not by analyzed\n')
            continue
        t = s.transpose((k*5)%12)

        fp_s = converter.freeze(s, fmt='pickle')
        fp_t = converter.freeze(t, fmt='pickle')

        with open('{}-parsed.txt'.format(composer), 'a') as output_file:
            output_file.write(fp_s+'\n')
        with open('{}-transposed.txt'.format(composer), 'a') as output_file:            
            output_file.write(fp_t+'\n')

In [141]:
s = converter.parse('/Volumes/S190813/Music_box/LTSM_Generative_Music/Ravel/ravel_xml/ravel_xml0.xml')

In [144]:
k = s.flat.keySignature.sharps

In [146]:
t= s.transpose((4+5)%12)

In [148]:
fp_s = converter.freeze(s, fmt='pickle')

In [149]:
fp_s

'/var/folders/0c/ktnszfs94_j5yczbmylqxwgc0000gn/T/music21/m21-d106897e58e9517954f9ea1fd770b8df.p'

In [152]:
with open('ravel_xml_list.txt', 'r') as f:
    mxllist = [line.strip() for line in tqdm(f.readlines())]
    
parse(mxllist, 'ravel')

In [45]:
mxllist

['ravel_xml0.xml',
 'ravel_xml1.xml',
 'ravel_xml2.xml',
 'ravel_xml3.xml',
 'ravel_xml4.xml',
 'ravel_xml5.xml']

In [153]:
import itertools
from collections import Counter

flatten = lambda l: [item for sublist in l for item in sublist]

In [154]:
# Define known chords
major, minor, suspended, augmented, diminished, major_sixth, minor_sixth, dominant_seventh, major_seventh, minor_seventh, half_diminished_seventh, diminished_seventh, major_ninth, dominant_ninth, dominant_minor_ninth, minor_ninth = [0,4,7],[0,3,7],[0,5,7],[0,4,8],[0,3,6],[0,4,7,9],[0,3,7,9],[0,4,7,10],[0,4,7,11],[0,3,7,10],[0,3,6,10],[0,3,6,9],[0,2,4,7,11],[0,2,4,7,10],[0,1,4,7,10],[0,2,3,7,10]
chord_types_list = [major, minor, suspended, augmented, diminished, major_sixth, minor_sixth, dominant_seventh, major_seventh, minor_seventh, half_diminished_seventh, diminished_seventh, major_ninth, dominant_ninth, dominant_minor_ninth, minor_ninth]
chord_types_string = ['major', 'minor', 'suspended', 'augmented', 'diminished', 'major_sixth', 'minor_sixth', 'dominant_seventh', 'major_seventh', 'minor_seventh', 'half_diminished_seventh', 'diminished_seventh', 'major_ninth', 'dominant_ninth', 'dominant_minor_ninth', 'minor_ninth']

roots = list(range(12))
chord_orders = flatten([[{(n+r)%12 for n in v} for v in chord_types_list] for r in roots])
unique_orders = []
for i in range(192):
    if chord_orders[i] not in unique_orders:
        unique_orders.append(chord_orders[i])

In [155]:
len(unique_orders)

151

In [156]:
def merge_chords(s):
    sf = s.flat
    chords_by_offset = []
    for i in range(int(sf.highestTime)):
        chords_by_offset.append(chord.Chord(sf.getElementsByOffset(i,i+1, includeEndBoundary=False, mustFinishInSpan=False, mustBeginInSpan=False).notes))
    return chords_by_offset

def find_neighbor_note(n, k):
    # find notes k steps away from n
    return (roots[n-6:]+roots[:(n+6)%12])[6+k], (roots[n-6:]+roots[:(n+6)%12])[6-k]

def find_note_distance(n1, n2):
    return abs(6 - (roots[n1-6:]+roots[:(n1+6)%12]).index(n2))

def find_chord_distance(set1, set2):
    d1, d2 = set1.difference(set2), set2.difference(set1)
    if len(d1) < len(d2):
        longer, shorter = d2, list(d1)
    else:
        longer, shorter = d1, list(d2)
    distances = []
    for combination in itertools.combinations(longer, len(shorter)):
        for permutation in itertools.permutations(combination):
            dist_p = abs(len(d1)-len(d2))*3 # length difference means notes need to be added/deleted. weighted by 3
            for i in range(len(shorter)):
                dist_p += find_note_distance(shorter[i], permutation[i])
            distances.append(dist_p)
    return min(distances)

In [93]:
find_neighbor_note(9,3)

(0, 6)

In [95]:
abs(6 - (roots[9-6:]+roots[:(9+6)%12]).index(6))

3

In [101]:
(roots[9-6:]+roots[:(9+6)%12])[6+3], (roots[9-6:]+roots[:(9+6)%12])[6-3]

(0, 6)

In [104]:
(roots[9-6:]+roots[:(9+6)%12])[6+3]

0

In [105]:
(roots[9-6:]+roots[:(9+6)%12])

[3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2]

In [119]:
roots[9-6:]

[3, 4, 5, 6, 7, 8, 9, 10, 11]

In [118]:
roots[:(9+6)%12]

[0, 1, 2]

In [120]:
(roots[9-6:]+roots[:(9+6)%12])[6-3]

6

In [125]:
roots[:(9+6)%12]

[0, 1, 2]

In [128]:
roots[9-6:]+roots[:(9-6)%12]

[3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2]

In [136]:
(roots[10-6:]+roots[:(10-6)%12])[6-3]

7

In [137]:
(roots[10-6:]+roots[:(10-6)%12])

[4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3]

In [157]:
CACHE = dict()

def find_closest_chord(c, cache=CACHE):
    if len(c) == 0:
        return -1 # use -1 for rest (chords are 0 to 191)
    
    # retrieve from existing knowledge
    o_str, o, p = str(c.normalOrder), set(c.normalOrder), c.pitchClasses
    if o in chord_orders:
        return chord_orders.index(o)
        # the above root sometimes differs from c.findRoot(), which might be more reliable.
        # however, the errors are rare and it should be good enough for now.
    if o_str in cache.keys():
        return cache[o_str]
    
    # find closest chord from scratch
    chord_distances = dict()
    most_common_note = Counter(c.pitchClasses).most_common(1)[0][0]

    for i in range(192):
        d = find_chord_distance(o, chord_orders[i])
        # prioritize found chord's root note if most common note of the chord.
        if int(i/16) == most_common_note:
            d += -1
        if chord_distances.get(d) == None:
            chord_distances[d] = []
        chord_distances[d].append(i)

    # if multiple chords are tied, use first one (could be better)
    closest_chord = chord_distances[min(chord_distances.keys())][0]
    
    cache[o_str] = closest_chord
    return closest_chord

In [158]:
def extract_features(parsed_list, idx):
    s = converter.thaw(parsed_list[idx])
    chords_by_offset = merge_chords(s)

    chord_sequence = []
    for i in range(len(chords_by_offset)):
        chord_sequence.append(find_closest_chord(chords_by_offset[i], CACHE))
    return chord_sequence

In [159]:
with open('ravel-parsed.txt', 'r') as f:
    FILES_RAVEL = [line.strip() for line in tqdm(f.readlines())]
    
for i in tqdm(range(len(FILES_RAVEL))):
    with open('ravel-chordsequence.txt', 'a') as f:
        f.write(str(extract_features(FILES_RAVEL, i))+'\n')

In [160]:
# Additional feature set: extract durations of notes, chords, and rests
def find_length_add_to_list(cnr, out_list):
    try:
        out_list.append(cnr.duration.fullName)
    except:
        out_list.append(str(cnr.duration.quarterLength))

def extract_cnr_duration(piece):
    s = converter.thaw(piece).flat
    chords, notes, rests = [], [], []
    for c in tqdm(s.getElementsByClass(chord.Chord)):
        find_length_add_to_list(c, chords)
    for n in tqdm(s.getElementsByClass(note.Note)):
        find_length_add_to_list(n, notes)
    for r in tqdm(s.getElementsByClass(note.Rest)):
        find_length_add_to_list(r, rests)
    elements = ['chord|'+d for d in tqdm(chords)] + ['note|'+d for d in tqdm(notes)] + ['rest|'+d for d in tqdm(rests)]
    return ';'.join(elements)

In [161]:
for piece in FILES_RAVEL:
    with open('ravel-durations.txt', 'a') as f:
        f.write(extract_cnr_duration(piece)+'\n')